In [1]:
import requests
from bs4 import BeautifulSoup
import traceback
import pandas as pd

In [2]:
code_num = ['032830','003690','082640','085620','088350']
code_name = ['삼성생명','코리안리','동양생명','미래에셋생명','한화생명']

In [3]:
def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None

In [4]:
#i는 위치, v는 code num
for i, v in enumerate(code_num):
    code = v
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
    res = requests.get(url)
    res.encoding = 'utf-8'
    res.status_code
    
    soap = BeautifulSoup(res.text, 'lxml')
    
    
    el_table_navi = soap.find("table", class_="Nnavi")
    el_td_last = el_table_navi.find("td", class_="pgRR")
    pg_last = el_td_last.a.get('href').rsplit('&')[1]
    pg_last = pg_last.split('=')[1]
    pg_last = int(pg_last)
    
    data = parse_page(code, 1)
    
    for j in range(2,62):
        data = pd.concat([data,pd.DataFrame(parse_page(code, j))],axis = 0)
        
    data.to_csv('%s.csv'%code_name[i])
    